In [1]:
import cv2
import numpy as np
import os

In [2]:
exam = '25080225_08_2019_08_37_59'
path_out = '{}/pupil_deep/eye_test/out'.format(os.getcwd().replace('/auxiliaries',''))

In [3]:
paths = []
if exam == '':
    path = '{}/pupil_deep/eye_test/out/exam'.format(os.getcwd().replace('/auxiliaries',''))
    files = os.listdir(path)
    paths = ['{}/{}'.format(path, x) for x in files]
else:
    paths.append('{}/pupil_deep/eye_test/out/{}'.format(os.getcwd().replace('/auxiliaries',''), exam))

In [4]:
paths

['/media/marcos/Dados/Projects/PupilDeep/pupil_deep/eye_test/out/25080225_08_2019_08_37_59']

In [5]:
number_files = []
os.listdir(paths[0])

for file in os.listdir(paths[0]):
    if 'original_' in file:
        number_files.append(file)

print('quantity: {}, examples: {}'.format(len(number_files), number_files[0: 10: 1]))

quantity: 10, examples: ['original_0.png', 'original_1.png', 'original_3.png', 'original_4.png', 'original_5.png', 'original_6.png', 'original_7.png', 'original_8.png', 'original_9.png', 'original_2.png']


In [6]:
img_array = []
size = None

for ex in paths:
    for i in range(1, len(number_files)):
        path_im_orig = 'original_{}.png'.format(i)

        path_im_proc = 'process_{}.png'.format(i)
        path_im_bin_pre = 'binary_pre_process_{}.png'.format(i)

        path_im_reflex = 'img_no_reflex_{}.png'.format(i)
        path_im_bin = 'binary_process_{}.png'.format(i)        

        path_im_final = 'final_{}.png'.format(i)

        im_orig = cv2.imread('{}/{}'.format(ex, path_im_orig))
        im_proc = cv2.imread('{}/{}'.format(ex, path_im_proc))
        im_bin_pre = cv2.imread('{}/{}'.format(ex, path_im_bin_pre))
        im_reflex = cv2.imread('{}/{}'.format(ex, path_im_reflex))
        im_bin = cv2.imread('{}/{}'.format(ex, path_im_bin))
        im_final = cv2.imread('{}/{}'.format(ex, path_im_final))        

        im_pres1 = cv2.hconcat([im_orig, im_proc, im_bin_pre])
        im_pres2 = cv2.hconcat([im_reflex, im_bin, im_final])
        im_pres = cv2.vconcat([im_pres1, im_pres2])

        img_array.append(im_pres)

        if size is None:
            height, width, _ = im_pres.shape
            size = (width,height)

In [7]:
if exam == '':
    exam = 'full_exams'

out = cv2.VideoWriter('{}/{}.mp4'.format(path_out, exam), cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()